#Clase Nodo

Atributos:
* el caracter del nodo
* lista de indices donde aparece
* si es o no un nodo final
* nivel
* lista de nodos hijos

In [5]:
class Nodo:
	def __init__ ( self, caracter, final ):
		self.caracter = caracter
		self.indices = []
		self.es_final = final
		self.level = 0

		self.hijos = []

#Clase Trie

In [4]:
# la clase del arbol.
# Su unico atributo es el nodo raiz
class Trie:
  def __init__(self):
    self.raiz = Nodo("#", False)
  
  # Función recursiva para insertar un sufijo en el trie
  # nodo: nodo actual donde insertar
  # cadena: el sufijo completo a insertar
  # i: índice actual en la cadena
  # og_i: índice original donde empieza este sufijo en la cadena original
  def insertar(self, nodo, cadena, i, og_i):
    length = len(cadena)
    
    # Caso base: si ya procesamos toda la cadena, terminar
    if i >= length:
      return
    
    # Buscar si ya existe un hijo con el carácter actual
    hijo_encontrado = None
    for h in nodo.hijos:
      if h.caracter == cadena[i]:
        hijo_encontrado = h
        break
    
    # Si encontramos el hijo, agregar el índice y continuar recursivamente
    if hijo_encontrado:
      hijo_encontrado.indices.append(og_i + i)
      self.insertar(hijo_encontrado, cadena, i+1, og_i)
    else:
      # Si no existe el hijo, crearlo
      es_ultimo = (i == length - 1)  # Es el último carácter del sufijo
      hijo = Nodo(cadena[i], es_ultimo)
      hijo.indices.append(og_i + i)
      hijo.level = nodo.level + 1
      
      # Agregar el hijo al nodo actual
      nodo.hijos.append(hijo)
      
      # Continuar insertando el resto de la cadena
      self.insertar(hijo, cadena, i+1, og_i)

  # Versión iterativa de la inserción
  def insertar_iter(self, cadena, index_sufijo):
    """
    Inserta un sufijo de manera iterativa
    cadena: el sufijo a insertar
    index_sufijo: índice donde empieza este sufijo en la cadena original
    """
    nodo_actual = self.raiz
    
    # Recorrer cada carácter del sufijo
    for i, caracter in enumerate(cadena):
      # Buscar si ya existe un hijo con este carácter
      hijo_encontrado = None
      for hijo in nodo_actual.hijos:
        if hijo.caracter == caracter:
          hijo_encontrado = hijo
          break
      
      # Si encontramos el hijo existente
      if hijo_encontrado:
        # Agregar el índice de aparición
        hijo_encontrado.indices.append(index_sufijo + i)
        # Movernos a este hijo para el siguiente carácter
        nodo_actual = hijo_encontrado
      else:
        # Crear nuevo hijo para este carácter
        es_final = (i == len(cadena) - 1)  # Es el último carácter
        nuevo_hijo = Nodo(caracter, es_final)
        nuevo_hijo.indices.append(index_sufijo + i)
        nuevo_hijo.level = nodo_actual.level + 1
        
        # Agregar el nuevo hijo al nodo actual
        nodo_actual.hijos.append(nuevo_hijo)
        
        # Movernos al nuevo hijo
        nodo_actual = nuevo_hijo

  # Función para buscar un patrón en el trie
  def buscar_patron(self, patron):
    """
    Busca un patrón en el suffix trie
    Retorna las posiciones donde aparece el patrón en la cadena original
    """
    nodo_actual = self.raiz
    
    # Navegar por cada carácter del patrón
    for caracter in patron:
      hijo_encontrado = None
      
      # Buscar el carácter en los hijos del nodo actual
      for hijo in nodo_actual.hijos:
        if hijo.caracter == caracter:
          hijo_encontrado = hijo
          break
      
      # Si no encontramos el carácter, el patrón no existe
      if not hijo_encontrado:
        return []
      
      # Movernos al hijo encontrado
      nodo_actual = hijo_encontrado
    
    # Si llegamos aquí, encontramos todo el patrón
    # Retornar todas las posiciones donde aparece
    return nodo_actual.indices

  # Función para encontrar el sufijo común más largo entre dos sufijos
  def lcs_sufijos(self, sufijo1, sufijo2):
    """
    Encuentra la longitud del prefijo común más largo entre dos sufijos
    """
    i = 0
    while (i < len(sufijo1) and i < len(sufijo2) and 
           sufijo1[i] == sufijo2[i]):
      i += 1
    return i

  # Recorrido in order para mostrar el arbol
  def show_inorder(self, nodo):
    print(f"\nSoy '{nodo.caracter}', existo en posiciones: {nodo.indices}, "
          f"¿es final?: {nodo.es_final}, nivel: {nodo.level}")
    if nodo.hijos:
      print(f"\tHijos: {[h.caracter for h in nodo.hijos]}")
    
    # Recorrer todos los hijos recursivamente
    for h in nodo.hijos:
      self.show_inorder(h)
  
  # Función para mostrar todos los sufijos almacenados
  def mostrar_sufijos(self, nodo=None, sufijo_actual=""):
    """
    Muestra todos los sufijos almacenados en el trie
    """
    if nodo is None:
      nodo = self.raiz
      print("=== SUFIJOS ALMACENADOS EN EL TRIE ===")
    
    # Si este nodo es final, imprimir el sufijo
    if nodo.es_final:
      print(f"Sufijo: '{sufijo_actual}' (posiciones: {nodo.indices})")
    
    # Recorrer todos los hijos
    for hijo in nodo.hijos:
      self.mostrar_sufijos(hijo, sufijo_actual + hijo.caracter)

In [6]:
# Ejemplo práctico: construir suffix trie para "anabanana"
palabra = "anabanana"
n = len(palabra)

# Generar todos los sufijos de la palabra
sufijos = [palabra[i:] for i in range(n)]
print("=== CADENA ORIGINAL ===")
print(f"Palabra: '{palabra}'")
print(f"Longitud: {n}")
print("\n=== SUFIJOS GENERADOS ===")
for i, sufijo in enumerate(sufijos):
    print(f"Sufijo {i}: '{sufijo}' (empieza en posición {i})")

print("\n" + "="*50)
print("CONSTRUYENDO EL SUFFIX TRIE...")
print("="*50)

# Crear el trie y insertar todos los sufijos
trie = Trie()

# Método 1: Usando inserción recursiva
print("\n--- Insertando sufijos con método RECURSIVO ---")
for i in range(len(sufijos)):
    print(f"Insertando sufijo {i}: '{sufijos[i]}'")
    trie.insertar(trie.raiz, sufijos[i], 0, i)

# Mostrar la estructura del trie
print("\n=== ESTRUCTURA DEL TRIE ===")
trie.show_inorder(trie.raiz)

# Mostrar todos los sufijos almacenados
print("\n")
trie.mostrar_sufijos()

# Probar búsquedas de patrones
print("\n" + "="*50)
print("PROBANDO BÚSQUEDAS DE PATRONES")
print("="*50)

patrones_prueba = ["ana", "ban", "na", "a", "xyz", "banana"]
for patron in patrones_prueba:
    posiciones = trie.buscar_patron(patron)
    if posiciones:
        print(f"✓ Patrón '{patron}' encontrado en posiciones: {posiciones}")
    else:
        print(f"✗ Patrón '{patron}' NO encontrado")

print("\n" + "="*50)
print("PROBANDO MÉTODO ITERATIVO")
print("="*50)

# Crear un nuevo trie para probar el método iterativo
trie_iter = Trie()
print("\n--- Insertando sufijos con método ITERATIVO ---")
for i, sufijo in enumerate(sufijos):
    print(f"Insertando sufijo {i}: '{sufijo}'")
    trie_iter.insertar_iter(sufijo, i)

print("\n=== VERIFICANDO QUE AMBOS MÉTODOS DAN EL MISMO RESULTADO ===")
for patron in ["ana", "ban", "na"]:
    pos_recursivo = trie.buscar_patron(patron)
    pos_iterativo = trie_iter.buscar_patron(patron)
    
    print(f"Patrón '{patron}':")
    print(f"  Recursivo: {pos_recursivo}")
    print(f"  Iterativo: {pos_iterativo}")
    print(f"  ¿Iguales?: {pos_recursivo == pos_iterativo}")
    print()

=== CADENA ORIGINAL ===
Palabra: 'anabanana'
Longitud: 9

=== SUFIJOS GENERADOS ===
Sufijo 0: 'anabanana' (empieza en posición 0)
Sufijo 1: 'nabanana' (empieza en posición 1)
Sufijo 2: 'abanana' (empieza en posición 2)
Sufijo 3: 'banana' (empieza en posición 3)
Sufijo 4: 'anana' (empieza en posición 4)
Sufijo 5: 'nana' (empieza en posición 5)
Sufijo 6: 'ana' (empieza en posición 6)
Sufijo 7: 'na' (empieza en posición 7)
Sufijo 8: 'a' (empieza en posición 8)

CONSTRUYENDO EL SUFFIX TRIE...

--- Insertando sufijos con método RECURSIVO ---
Insertando sufijo 0: 'anabanana'
Insertando sufijo 1: 'nabanana'
Insertando sufijo 2: 'abanana'
Insertando sufijo 3: 'banana'
Insertando sufijo 4: 'anana'
Insertando sufijo 5: 'nana'
Insertando sufijo 6: 'ana'
Insertando sufijo 7: 'na'
Insertando sufijo 8: 'a'

=== ESTRUCTURA DEL TRIE ===

Soy '#', existo en posiciones: [], ¿es final?: False, nivel: 0
	Hijos: ['a', 'n', 'b']

Soy 'a', existo en posiciones: [0, 2, 4, 6, 8], ¿es final?: False, nivel: 1
	H

# Análisis y Aplicaciones del Suffix Trie

## Complejidad:
- **Construcción**: O(n²) tiempo y espacio, donde n = longitud de la cadena
- **Búsqueda de patrón**: O(m) donde m = longitud del patrón
- **Espacio**: O(n²) en el peor caso

## Ventajas:
- Búsqueda rápida de patrones
- Útil para encontrar subcadenas repetidas
- Permite análisis eficiente de sufijos

## Aplicaciones:
- Búsqueda de patrones en texto
- Análisis de secuencias de ADN
- Compresión de datos
- Detección de plagios

In [7]:
# Ejemplo adicional: Análisis de una secuencia más compleja
print("="*60)
print("EJEMPLO AVANZADO: ANÁLISIS DE SECUENCIA COMPLEJA")
print("="*60)

# Probar con una secuencia que tenga más repeticiones
secuencia_compleja = "abcabcabc"
print(f"\nAnalizando la secuencia: '{secuencia_compleja}'")

# Crear suffix trie para la secuencia compleja
trie_complejo = Trie()
sufijos_complejos = [secuencia_compleja[i:] for i in range(len(secuencia_compleja))]

print(f"\nSufijos de '{secuencia_compleja}':")
for i, suf in enumerate(sufijos_complejos):
    print(f"  {i}: '{suf}'")

# Construir el trie
for i, sufijo in enumerate(sufijos_complejos):
    trie_complejo.insertar_iter(sufijo, i)

# Buscar patrones repetitivos
patrones_interes = ["abc", "bc", "ab", "c", "ca"]
print(f"\nBúsqueda de patrones repetitivos:")
for patron in patrones_interes:
    posiciones = trie_complejo.buscar_patron(patron)
    if posiciones:
        frecuencia = len(set(posiciones))  # Eliminar duplicados si los hay
        print(f"  '{patron}': aparece en posiciones {posiciones} (frecuencia: {frecuencia})")
    else:
        print(f"  '{patron}': no encontrado")

print(f"\nEstructura del trie para '{secuencia_compleja}':")
trie_complejo.show_inorder(trie_complejo.raiz)

EJEMPLO AVANZADO: ANÁLISIS DE SECUENCIA COMPLEJA

Analizando la secuencia: 'abcabcabc'

Sufijos de 'abcabcabc':
  0: 'abcabcabc'
  1: 'bcabcabc'
  2: 'cabcabc'
  3: 'abcabc'
  4: 'bcabc'
  5: 'cabc'
  6: 'abc'
  7: 'bc'
  8: 'c'

Búsqueda de patrones repetitivos:
  'abc': aparece en posiciones [2, 5, 8] (frecuencia: 3)
  'bc': aparece en posiciones [2, 5, 8] (frecuencia: 3)
  'ab': aparece en posiciones [1, 4, 7] (frecuencia: 3)
  'c': aparece en posiciones [2, 5, 8] (frecuencia: 3)
  'ca': aparece en posiciones [3, 6] (frecuencia: 2)

Estructura del trie para 'abcabcabc':

Soy '#', existo en posiciones: [], ¿es final?: False, nivel: 0
	Hijos: ['a', 'b', 'c']

Soy 'a', existo en posiciones: [0, 3, 6], ¿es final?: False, nivel: 1
	Hijos: ['b']

Soy 'b', existo en posiciones: [1, 4, 7], ¿es final?: False, nivel: 2
	Hijos: ['c']

Soy 'c', existo en posiciones: [2, 5, 8], ¿es final?: False, nivel: 3
	Hijos: ['a']

Soy 'a', existo en posiciones: [3, 6], ¿es final?: False, nivel: 4
	Hijos: [

In [8]:
# Función adicional: Encontrar la subcadena repetida más larga
def encontrar_subcadena_repetida_mas_larga(trie, palabra):
    """
    Encuentra la subcadena que aparece más veces en la palabra
    usando el suffix trie construido
    """
    print("\n" + "="*50)
    print("BUSCANDO SUBCADENAS REPETIDAS MÁS LARGAS")
    print("="*50)
    
    mejor_patron = ""
    mejor_frecuencia = 0
    mejor_posiciones = []
    
    # Probar subcadenas de diferentes longitudes
    for longitud in range(1, len(palabra)):
        for inicio in range(len(palabra) - longitud + 1):
            patron = palabra[inicio:inicio + longitud]
            posiciones = trie.buscar_patron(patron)
            
            # Contar apariciones únicas (sin solapamiento)
            if posiciones:
                frecuencia = len(set(posiciones))
                if frecuencia > mejor_frecuencia and frecuencia > 1:
                    mejor_patron = patron
                    mejor_frecuencia = frecuencia
                    mejor_posiciones = posiciones
    
    if mejor_patron:
        print(f"Subcadena más repetida: '{mejor_patron}'")
        print(f"Frecuencia: {mejor_frecuencia}")
        print(f"Posiciones: {mejor_posiciones}")
        
        # Mostrar las apariciones en contexto
        print(f"Contexto en '{palabra}':")
        for pos in set(mejor_posiciones):
            if pos + len(mejor_patron) <= len(palabra):
                contexto_inicio = max(0, pos - 2)
                contexto_fin = min(len(palabra), pos + len(mejor_patron) + 2)
                contexto = palabra[contexto_inicio:contexto_fin]
                marcado = (contexto[:pos-contexto_inicio] + 
                          "[" + mejor_patron + "]" + 
                          contexto[pos-contexto_inicio+len(mejor_patron):])
                print(f"  Posición {pos}: ...{marcado}...")
    else:
        print("No se encontraron subcadenas repetidas")

# Probar con nuestros ejemplos
encontrar_subcadena_repetida_mas_larga(trie, palabra)
encontrar_subcadena_repetida_mas_larga(trie_complejo, secuencia_compleja)


BUSCANDO SUBCADENAS REPETIDAS MÁS LARGAS
Subcadena más repetida: 'a'
Frecuencia: 5
Posiciones: [0, 2, 4, 6, 8]
Contexto en 'anabanana':
  Posición 0: ...[a]na...
  Posición 2: ...an[a]ba...
  Posición 4: ...ab[a]na...
  Posición 6: ...an[a]na...
  Posición 8: ...an[a]...

BUSCANDO SUBCADENAS REPETIDAS MÁS LARGAS
Subcadena más repetida: 'a'
Frecuencia: 3
Posiciones: [0, 3, 6]
Contexto en 'abcabcabc':
  Posición 0: ...[a]bc...
  Posición 3: ...bc[a]bc...
  Posición 6: ...bc[a]bc...


# Resumen del Suffix Trie Completado

## ✅ Funciones Implementadas:

1. **`insertar()`** - Versión recursiva para insertar sufijos
2. **`insertar_iter()`** - Versión iterativa para insertar sufijos  
3. **`buscar_patron()`** - Busca un patrón y retorna sus posiciones
4. **`show_inorder()`** - Muestra la estructura del trie
5. **`mostrar_sufijos()`** - Lista todos los sufijos almacenados
6. **`encontrar_subcadena_repetida_mas_larga()`** - Análisis avanzado

## 🎯 Casos de Uso Demostrados:

- Construcción de suffix trie para cadenas simples y complejas
- Búsqueda eficiente de patrones
- Análisis de frecuencias de subcadenas
- Identificación de repeticiones en secuencias

## 📊 Rendimiento:
- **Construcción**: O(n²) 
- **Búsqueda**: O(m) donde m = longitud del patrón
- **Espacio**: O(n²) donde n = longitud de la cadena original